In [1]:
from models.Unet import UNet
from dataset.data import BatchMaker
import torch
import yaml
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.metrics import (
    jaccard_score,
    average_precision_score
)


In [22]:
BATCH = 1
#path_to_config = '/media/marcin/Dysk lokalny/Programowanie/Python/Magisterka/Praca Dyplomowa/noisy_labels/Kod/config/config.yaml'
path_to_config = '/media/cal314-1/9E044F59044F3415/Marcin/noisy_labels/Kod/config/config_lab.yaml'
with open(path_to_config, 'r') as config_file:
    config = yaml.safe_load(config_file)
model_path = config['save_model_path'] + '/mixedGT1_best_model'


annotator1 = BatchMaker(config_path=path_to_config, batch_size=BATCH, mode = 'all',segment = 'mixed',annotator= 1)
train_GT1 = annotator1.train_loader
annotator2 = BatchMaker(config_path=path_to_config, batch_size=BATCH, mode = 'all',segment = 'mixed',annotator= 2)
train_GT2 = annotator2.train_loader


/media/cal314-1/9E044F59044F3415/Marcin/noisy_labels/Dataset/SegSperm/train
/media/cal314-1/9E044F59044F3415/Marcin/noisy_labels/Dataset/SegSperm/test_small
/media/cal314-1/9E044F59044F3415/Marcin/noisy_labels/Dataset/SegSperm/test
/media/cal314-1/9E044F59044F3415/Marcin/noisy_labels/Dataset/SegSperm/train
/media/cal314-1/9E044F59044F3415/Marcin/noisy_labels/Dataset/SegSperm/test_small
/media/cal314-1/9E044F59044F3415/Marcin/noisy_labels/Dataset/SegSperm/test


In [32]:
import matplotlib.pyplot as plt 

labels1 = []
labels2 = []
for i in range(len(train_GT1.dataset)):
    labels1.append(train_GT1.dataset.__getitem__(i)[1][0])
    labels2.append(train_GT2.dataset.__getitem__(i)[1][0])



In [33]:
label1np = np.array(labels1)
label2np = np.array(labels2)

In [48]:
#IoU = jaccard_score(label1np, label2np)
classes = np.unique(np.concatenate((label1np.flatten(), label2np.flatten())))
jaccard_scores = []
#average_precision = average_precision_score(label1np.flatten(), label2np.flatten())


for cls in classes:
    true_mask = (label2np.flatten() == cls)
    pred_mask = (label1np.flatten() == cls)

    intersection = np.sum(true_mask & pred_mask)
    union = np.sum(true_mask | pred_mask)

    jaccard = intersection / union if union > 0 else 0
    jaccard_scores.append(jaccard)

# Average the Jaccard scores across all classes
average_jaccard = np.mean(jaccard_scores)

print("Jaccard scores for each class:", jaccard_scores)

#print("IoU: {}".format(IoU))
#print("Average Precision: {}".format(average_precision))    

Jaccard scores for each class: [0.9939750840052808, 0.722969509149901, 0.3689429403379839]


In [53]:
import tensorflow as tf

score = tf.keras.metrics.MeanIoU(num_classes=3)
score.update_state(label1np.flatten(), label2np.flatten())
score.result().numpy()

1.0